# Chapter 3: Spins and Qubits

A qubit has a hidden spin state (described by a unit vector in R<sup>2</sup>). Because of the uncertainly principle, we cannot directly observe this state without changing it. When we measure it, we observe either "0" or "1" one of the states with a certain probability (given by teh direction of the measurement and the state of the Qubit).

1. Choose a direction of measurement. This is akin to creating a `Measurement` object, by passing it an `OrderedOrthonormalBases`
2. Create a Qubit, using the `Qubit` class
3. Measure the qubit using the measurement : `out = q0.get_measured(msmt)`. This returns a 0 or a 1, and changes the state of qubit

We will look at more detains in the next few sections

## Measurement

The measuring process involves an orthonormal matrix (direction of measurement) and the qubit (with its unobservable spin state, a Ket in R<sup>2</sup>)

Consider the measurement matrix:

$$A = \begin{bmatrix} a & c \\ b & d \end{bmatrix}$$

and a qubit with state vector 
$$v = \begin{bmatrix} x \\ y \end{bmatrix}$$

We multiply $A^T * v$

$$d = \begin{bmatrix} ax + by \\ cx + dy \end{bmatrix}$$

Or we can express `v` in terms of the basis vectors as:
$$v = (ax + by) * \begin{bmatrix} a\\ b \end{bmatrix} + (cx + dy) * \begin{bmatrix} c\\ d \end{bmatrix}$$

Lets call these weights $w_0$ and $w_1$, ie $w_0 = ax+by$ and $w_1 = cx+dy$

Then with probability $w_0^2$ the qubit is measured as being in state 0 and its internal state flips to $\begin{bmatrix} a  \\ b \end{bmatrix}$ (the first direction of measurement). But also there is a probability $w_1^2$ with which the qubit is measured to be 1 and its state flips to $\begin{bmatrix} c  \\ d \end{bmatrix}$ (the second direction of measurement)

### Angle of measurement
For a certain angle `x` (in degrees), we can create the measurement object like so:
`msmt = Measurement(theta = deg_to_rad(x))`

The formula for the matrix is given by:
$$M = \begin{bmatrix} cos(x/2) & sin(x/2) \\ -sin(x/2) & cos(x/2) \end{bmatrix}$$


## Measuring a qubit multiple times in the same direction

If we measure the qubit in one direction, its state flips to one of the basis vector directions of the measurement. Therefore on second (and furthur) measurements in the same direction (using the same measurement matrix), we will keep getting back the same bit we measured, as there is no probability amplitude in the other direction

$$M = \begin{bmatrix} a & c \\ b & d \end{bmatrix}, a^2 + b^2 = 1, ac+bd = 0, c^2+d^2=1$$
$$q = \begin{bmatrix} x \\ y \end{bmatrix}$$

$$ m = M^T*q = \begin{bmatrix} ax+by \\ cx+dy \end{bmatrix} $$
Now with probability $(ax+by)^2$ the qubit's state changes to $\begin{bmatrix} a  \\ b  \end{bmatrix}$, or state `0`

Now for the next measurement:
$$q = \begin{bmatrix} a \\ b \end{bmatrix}$$

$$M^T * q = \begin{bmatrix} a^2+b^2 \\ ca+db \end{bmatrix} $$
But $M$ was orthonormal, so the newly measured value is:
$$M^T * q = \begin{bmatrix} 1 \\ 0 \end{bmatrix} $$

Thus with probability `1`, we get the earlier state


In [3]:
from basics import Qubit, Measurement
from utils import deg_to_rad

msmt = Measurement(theta = deg_to_rad(90))
num_expts = 100
q0 = Qubit()
for i in range(num_expts):
    out = q0.get_measured(msmt)
    if i == 0:
        res = out
    else:
        assert out == res # the result should never change if we measure later using the same measurement matrix

## Measuring multiple qubits in a direction

Since measurement is probabilistic, we need to perform multiple experiments to get the mean values.

In this experiment, we fix a direction of measurement, and then measure multiple different qubits with it. Given the measurement process described above, we expect "0" 50% of the time for any direction



In [9]:
from basics import Qubit, Measurement
from utils import deg_to_rad, is_close

for direction in [0, 45, 90]:
    print(f'Measuring in direction', direction)
    msmt = Measurement(theta = deg_to_rad(direction))  # what if we have other angles?
    num_zero = 0
    num_expts = 1000
    for i in range(num_expts):
        q0 = Qubit()
        out = q0.get_measured(msmt)
        if out == 0:
            num_zero += 1
    print (num_zero / num_expts)
    # Any direction we choose, when we measure a random bunch of qubits, they will be 0 (or 1) 50% of the time

Measuring in direction 0
0.518
Measuring in direction 45
0.529
Measuring in direction 90
0.518


## Measuring a qubit twice in 2 different directions
Lets first measure a qubit at 0 degree. Say its state after that measurement is '0' or $\begin{bmatrix} 1 \\ 0  \end{bmatrix}$


### First 0, then 90

After measuring at 0 degree, say we measure it at 90 degree, which corresponds to the measurement matrix:
$$\begin{bmatrix} 1/\sqrt 2  & 1/\sqrt 2  \\ -1/\sqrt 2  & 1/\sqrt 2  \end{bmatrix}$$

measuring the qubit with this matrix gives us:
$$\begin{bmatrix} 1/\sqrt 2  & -1/\sqrt 2  \\ 1/\sqrt 2  & 1/\sqrt 2  \end{bmatrix} * \begin{bmatrix} 1 \\ 0  \end{bmatrix} = \begin{bmatrix} 1/\sqrt 2 \\ 1/\sqrt 2  \end{bmatrix}$$

Squaring the numbers we see there is `0.5` probability for either bit

### First 0, then 60
For 60 degree, the measurement matrix is:
$$\begin{bmatrix} \sqrt 3 / 2  & 1/ 2  \\ -1/ 2  & \sqrt 3 / 2  \end{bmatrix}$$measuring the qubit with this matrix gives us:
$$\begin{bmatrix} \sqrt 3 / 2  & 1/ 2  \\ -1/ 2  & \sqrt 3 / 2  \end{bmatrix} * \begin{bmatrix} 1 \\ 0  \end{bmatrix} = \begin{bmatrix}  \sqrt 3 / 2 \\ 1/2  \end{bmatrix}$$

Squaring the amplitudes, we see at this angle, we have 0.75 chance for bit 0, and 0.25 chance for bit 1

The following code snippet shows the above simulation



In [8]:
for degree in [90, 60]:
    msmt0 = Measurement(theta = deg_to_rad(0))
    msmt_rotate = Measurement(theta = deg_to_rad(degree))
    num_expts = 1000
    num_zero = 0
    counts = {}
    for i in range(num_expts):
        q0 = Qubit()
        out0 = q0.get_measured(msmt0)
        out1 = q0.get_measured(msmt_rotate)
        counts[(out0, out1)] = counts.get((out0, out1), 0) + 1
        if out1 == 0:
            num_zero += 1
    # probability that it is 0 after second msmt, given it was 0 after first msmt
    p_m1_0_m0_0 = counts[(0,0)] / (counts[(0,0)] + counts[(0,1)])
    print(f'measure at 0, then {degree}', p_m1_0_m0_0)

measure at 0, then 90 0.5121951219512195
measure at 0, then 60 0.7409638554216867


## Communication

Lets say we have Alice and Bob who want to communicate, and Eve wants to evesdrop. Let Alice choose measurement bases $ (\ket{a_0}, \ket{a_1} )$, and Bob chooses $ (\ket{b_0}, \ket{b_1} )$. Therefore whatever qubit Alice transmits will be either in state \ket{a_0} or \ket{a_1}. When she wants to send a 0, she sends a qubit that has been measured 0 (and thus has state \ket{a_0}). Bob will measure it and get: $\ket{a_0} = d_0\ket{b_0} + d_1\ket{b_1}$. Therefore with probability $d_0^2$, he will correctly read the 0 bit from Alice as 0.

This transmission sounds noisy, however the crux of the matter is, if Eve intercepts the qubit sent from Alice and tries to read it, it will change the state, and that could be detected.

### BB84 protocol
Finally an interesting application. Alice and Bob wants to exchange a key for future cryptographic exchanges, but they want to detect if someone has eavesdropped on the transmitted message

We start with 2 bases, standard (vertical) and horizontal:

$$V = \begin{bmatrix} 1  & 0  \\ 0  & 1  \end{bmatrix}$$
$$H = \begin{bmatrix} 1/\sqrt 2  & 1/\sqrt 2  \\ -1/\sqrt 2  & 1/\sqrt 2  \end{bmatrix}$$

1. Alice chooses a long key (binary string of 0 and 1s) of length $4n$
2. For each bit she chooses $H$ or $V$ equiprobably, and then she sends Bob the corresponding qubit. For example if her bit is `1` and she choose $V$, she will measure a qubit with $V$, till she observes 1 (internal state $\begin{bmatrix} 0  \\ 1  \end{bmatrix}$), and then send this to Bob
3. Bob chooses a $H$ or $V$ equiprobably and measures the qubit Alice sent
4. Probabilistically, Half the time Alice and Bob will end up using the same measurement basis (and they will then get the same exact bit), while in the other half of cases Bob will guess the right bit from Alice with `0.5` probability. Alice and Bob compare their choice of $V$ and $H$, and keep the observed bits where they used the same measurement. They expect around half ($2n$) bits to remain after this step

#### Detecting Eve

Eve can only guess the basis being used. Thus in the $2n$ bits where Alice and Bob chose the same basis (after step 4 above), Eve will be right half the time (and all three will get the same bit), but the other times, she will change the qubit's state, and thus when Bob measures it he will only get the bit as Alice half the time.

Now of the remaining $2n$ bits, Alice and Bob compare $n$ of them over an unencrypted line. If the match exactly, Eve wasnt there, and they use the leftover $n$bits as their secret key. However if they dont match, they have successfully detected Eve listening in.

In [17]:
V = Measurement(theta = deg_to_rad(0))
H = Measurement(theta = deg_to_rad(90))
import random

for eve in [True, False]:
    
    # Alice
    n = 1024
    alice_msmt = []
    # Alice decides a key
    msg_to_send = [int(random.random() < 0.5) for i in range(4*n)] # A binary string of length 4n
    qubit_to_send = []
    for bit_to_send in msg_to_send:
        #Alice decides a msmt base for each bit
        rnd = random.random() < 0.5
        msmt = V if rnd else H # Alice picks a random measurement base
        alice_msmt += [('h','v')[rnd]]
        while True:
            q = Qubit()
            out = q.get_measured(msmt)
            if out == bit_to_send:
                break
        qubit_to_send += [q]


    if eve:
        
    
    
    # Bob received qubit_to_send
    qubits_received_by_bob = qubit_to_send
    bob_msmt = []
    msg_read_by_bob = []
    for q in qubits_received_by_bob:
        rnd = random.random() < 0.5
        msmt = V if rnd else H # Bob picks a random measurement base
        bob_msmt += [('h','v')[rnd]]
        msg_read_by_bob += [q.get_measured(msmt)]
    
    
    # Now Alice and Bob compare their bases:
    matching_bases = [i==j for i, j in zip(alice_msmt, bob_msmt)]
    
    
    #Alice throws out non matching ones:
    filtered_msg_alice = [i for i, match in zip(msg_to_send, matching_bases) if match]
    
    #Bob throws out non matching ones:
    filtered_msg_bob = [i for i, match in zip(msg_read_by_bob, matching_bases) if match]
    
    # Now they match half of the remaining messages:
    nbits_to_check = len(filtered_msg_alice)//2
    msg_from_alice = filtered_msg_alice[:nbits_to_check]
    msg_from_bob = filtered_msg_bob[:nbits_to_check]
    
    perc_match = sum([i==j for i, j in zip(msg_from_alice, msg_from_bob)]) / len(msg_from_bob)
    print('Percentage match', perc_match)
    
    if perc_match == 1.0:
        alice_key = filtered_msg_alice[nbits_to_check:]
        bob_key = filtered_msg_bob[nbits_to_check:]
        print('No Eve detected, key transfer successful')
    else:
        print('Eve detected')

Percentage match 1.0
No Eve detected, key transfer successful
Percentage match 1.0
No Eve detected, key transfer successful
